# Llama-Index with OpenRouter Function Calling Demo

This notebook demonstrates how to use llama-index with OpenRouter to create a function calling agent.

## Setup

First, let's import the necessary libraries and set up our environment.

In [12]:
import os
from typing import List, Dict, Any
from dotenv import load_dotenv
from llama_index.llms.openrouter import OpenRouter
from llama_index.core import Settings
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load environment variables
load_dotenv()

# Get API key from environment
api_key = os.getenv("OPENROUTER_API_KEY")
if not api_key:
    raise ValueError("OPENROUTER_API_KEY environment variable is not set")

# Set up a local embedding model (in case we want to use vector embeddings later)
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

## Define Functions

Let's define some example functions that our agent can call.

In [13]:
def search_weather(location: str) -> str:
    """
    Search for the current weather in a given location.
    
    Args:
        location: The city or location to get weather for
        
    Returns:
        A string describing the weather
    """
    # This is a mock function - in a real application, you would call a weather API
    return f"The weather in {location} is currently sunny with a temperature of 72°F."

def calculate_mortgage(principal: float, interest_rate: float, years: int) -> str:
    """
    Calculate monthly mortgage payment.
    
    Args:
        principal: The loan amount in dollars
        interest_rate: Annual interest rate (as a percentage, e.g., 5.5 for 5.5%)
        years: Loan term in years
        
    Returns:
        A string with the monthly payment amount
    """
    monthly_rate = interest_rate / 100 / 12
    num_payments = years * 12
    monthly_payment = principal * (monthly_rate * (1 + monthly_rate) ** num_payments) / ((1 + monthly_rate) ** num_payments - 1)
    return f"For a ${principal:,.2f} loan with {interest_rate}% interest over {years} years, the monthly payment would be ${monthly_payment:,.2f}"

# Create function tools from our functions
weather_tool = FunctionTool.from_defaults(fn=search_weather)
mortgage_tool = FunctionTool.from_defaults(fn=calculate_mortgage)

## Initialize OpenRouter LLM

Now let's initialize the OpenRouter LLM and set it as the default for llama-index.

In [14]:
# Initialize the OpenRouter LLM
# Using a model that supports function calling
llm = OpenRouter(
    api_key=api_key,
    model="anthropic/claude-3-opus-20240229",  # You can change to another model that supports function calling
)

# Set the LLM and embedding model as the defaults for llama-index
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

## Create a ReAct Agent

Let's create a ReAct agent with our tools.

In [15]:
# Create a ReAct agent with our tools
agent = ReActAgent.from_tools(
    [weather_tool, mortgage_tool],
    llm=llm,
    verbose=True,
)

## Test the Agent

Now let's test our agent with some example queries.

In [16]:
# Test weather query
response = agent.chat("What's the weather in San Francisco?")
print(f"Agent response: {response}")

> Running step e79ac1f1-1b2a-440a-af61-b4d73fd64b83. Step input: What's the weather in San Francisco?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_weather
Action Input: {'location': 'San Francisco'}
Observation: The weather in San Francisco is currently sunny with a temperature of 72°F.
> Running step a1f7769d-3525-4652-add7-5f5edf3fb9bf. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The weather in San Francisco is currently sunny with a temperature of 72°F.
Agent response: The weather in San Francisco is currently sunny with a temperature of 72°F.


In [6]:
# Test mortgage calculation query
response = agent.chat("Calculate mortgage payment for a $500,000 loan at 4.5% for 30 years")
print(f"Agent response: {response}")

> Running step c05e536e-b720-4303-af94-481b0917a61c. Step input: Calculate mortgage payment for a $500,000 loan at 4.5% for 30 years
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: calculate_mortgage
Action Input: {'principal': 500000, 'interest_rate': 4.5, 'years': 30}
Observation: For a $500,000.00 loan with 4.5% interest over 30 years, the monthly payment would be $2,533.43
> Running step e479c718-4191-40c5-8b1d-41ca1f54c51b. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Based on the provided loan details of a $500,000 principal amount, 4.5% annual interest rate, and a 30-year term, your monthly mortgage payment would be $2,533.43. This amount includes both the principal and interest portions of the loan, but does not include property taxes, insurance, or any other fees that may be part of your total monthly payment. Keep in mind that this is just

## Try Your Own Queries

Now you can try your own queries to see how the agent responds.

In [7]:
# Try your own query
your_query = "What's the weather in Tokyo?"
response = agent.chat(your_query)
print(f"Query: {your_query}")
print(f"Agent response: {response}")

> Running step 8ab1ff21-fbff-4675-bf1c-02650ea35c42. Step input: What's the weather in Tokyo?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: search_weather
Action Input: {'location': 'Tokyo'}
Observation: The weather in Tokyo is currently sunny with a temperature of 72°F.
> Running step f7888f99-878f-4189-a723-875220b555c4. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The current weather in Tokyo is sunny with a temperature of 72°F.
Query: What's the weather in Tokyo?
Agent response: The current weather in Tokyo is sunny with a temperature of 72°F.


## Document Retrieval Example

Now let's demonstrate how to combine function calling with document retrieval.

In [8]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.tools import QueryEngineTool

# Create some sample documents for our knowledge base
documents = [
    Document(text="Python is a high-level, interpreted programming language known for its readability and versatility. It was created by Guido van Rossum and first released in 1991."),
    Document(text="JavaScript is a programming language that is one of the core technologies of the World Wide Web. It was created by Brendan Eich in 1995 while he was working at Netscape Communications Corporation."),
    Document(text="Rust is a multi-paradigm, general-purpose programming language designed for performance and safety, especially safe concurrency. It was created at Mozilla Research by Graydon Hoare in 2010."),
    Document(text="TypeScript is a programming language developed and maintained by Microsoft. It is a strict syntactical superset of JavaScript and adds optional static typing to the language. It was designed by Anders Hejlsberg in 2012."),
]

# Create a vector index from our documents
index = VectorStoreIndex.from_documents(documents)

# Create a query engine from the index
query_engine = index.as_query_engine()

# Create a query engine tool
query_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="programming_language_kb",
    description="Useful for answering questions about programming languages like Python, JavaScript, Rust, and TypeScript.",
)

In [9]:
# Create a new agent with both function tools and the query engine tool
combined_agent = ReActAgent.from_tools(
    [weather_tool, mortgage_tool, query_tool],
    llm=llm,
    verbose=True,
)

In [10]:
# Test a query about programming languages
programming_query = "When was Python created and who created it?"
response = combined_agent.chat(programming_query)
print(f"Query: {programming_query}")
print(f"Agent response: {response}")

> Running step 98a88201-5c51-4ce0-95bf-9c4c184e02e9. Step input: When was Python created and who created it?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: programming_language_kb
Action Input: {'input': 'When was Python created and who created it?'}
Observation: Python was created by Guido van Rossum and first released in 1991.
> Running step 01640514-8b5a-43d6-9371-297976dc442c. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Python was created by Guido van Rossum and first released in 1991.
Query: When was Python created and who created it?
Agent response: Python was created by Guido van Rossum and first released in 1991.


In [11]:
# Test a mixed query that requires both knowledge retrieval and function calling
mixed_query = "What's the weather in New York and when was JavaScript created?"
response = combined_agent.chat(mixed_query)
print(f"Query: {mixed_query}")
print(f"Agent response: {response}")

> Running step 1c09b75a-c0cb-4263-afe1-c15416e5a876. Step input: What's the weather in New York and when was JavaScript created?
Thought: The current language of the user is: English. I need to use tools to help me answer the question about the weather in New York and when JavaScript was created.
Action: search_weather
Action Input: {'location': 'New York'}
Observation: The weather in New York is currently sunny with a temperature of 72°F.
> Running step a44b43b1-dad6-4194-aecd-749a64be8c68. Step input: None
Thought: I have the weather information for New York, but I still need to find out when JavaScript was created. I'll use the programming language knowledge base for that.
Action: programming_language_kb
Action Input: {'input': 'When was JavaScript created?'}
Observation: JavaScript was created in 1995 by Brendan Eich while he was working at Netscape Communications Corporation.
> Running step aae57bbd-b467-4e57-b2b8-8c2959c9173d. Step input: None
Thought: I can answer without using 

## Conclusion

This notebook demonstrated how to use llama-index with OpenRouter to create a function calling agent and combine it with document retrieval. You can extend this example by adding more functions, using different models, or integrating with other llama-index features.